# Function Approximation: Scaling RL to the Real World

Welcome to the bridge between classic RL and deep RL! This notebook explains why we need function approximation and how it transforms RL.

## What You'll Learn

By the end of this notebook, you'll understand:
- The curse of dimensionality (with a library analogy!)
- Why tabular methods break at scale
- Function approximation: from memorization to generalization
- Linear vs neural network approximation
- The deadly triad: why combining these things is dangerous
- How to train Q-networks with gradient descent

**Prerequisites:** Classic algorithms section, neural network basics

**Time:** ~30 minutes

---
## The Big Picture: The Library Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE LIBRARY ANALOGY: WHY TABLES DON'T SCALE          │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine you're building a library of book reviews:           │
    │                                                                │
    │  TABULAR APPROACH (Q-Table):                                  │
    │    📚 Store a separate review for EVERY possible book        │
    │       "The Great Gatsby" → "★★★★★ Great American novel"      │
    │       "1984" → "★★★★★ Chilling dystopia"                     │
    │       "Harry Potter 1" → "★★★★☆ Magical adventure"           │
    │       ...                                                     │
    │                                                                │
    │    PROBLEM: There are MILLIONS of books!                      │
    │    You can't review every single one!                         │
    │                                                                │
    │  FUNCTION APPROXIMATION APPROACH:                             │
    │    🧠 Learn PATTERNS that predict good reviews:               │
    │       "Books with compelling characters → Higher rating"      │
    │       "Books by award-winning authors → Usually good"         │
    │       "Books in genres I like → Higher predicted rating"      │
    │                                                                │
    │    Now you can predict ratings for books you've NEVER read!  │
    │    This is GENERALIZATION!                                    │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

**The same principle applies to RL: Instead of memorizing Q-values for every state-action pair, we learn patterns that generalize!**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Rectangle, Circle, FancyArrowPatch

# Visualize tabular vs function approximation
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Tabular (memorization)
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.axis('off')
ax1.set_title('TABULAR Q-LEARNING\n"Memorization"', fontsize=14, fontweight='bold', color='#d32f2f')

# Draw a table
for row in range(5):
    for col in range(4):
        rect = Rectangle((1 + col*2, 6.5 - row), 1.8, 0.8, 
                         facecolor='#ffcdd2' if row == 0 else '#fff',
                         edgecolor='black', linewidth=1)
        ax1.add_patch(rect)

# Table headers
ax1.text(2, 7.1, 'State', ha='center', fontsize=9, fontweight='bold')
ax1.text(4, 7.1, 'UP', ha='center', fontsize=9, fontweight='bold')
ax1.text(6, 7.1, 'DOWN', ha='center', fontsize=9, fontweight='bold')
ax1.text(8, 7.1, 'LEFT', ha='center', fontsize=9, fontweight='bold')

# Table content
states = ['(0,0)', '(0,1)', '(0,2)', '...']
for i, s in enumerate(states):
    ax1.text(2, 6.1 - i, s, ha='center', fontsize=9)
    ax1.text(4, 6.1 - i, f'{np.random.randn():.1f}', ha='center', fontsize=8)
    ax1.text(6, 6.1 - i, f'{np.random.randn():.1f}', ha='center', fontsize=8)
    ax1.text(8, 6.1 - i, f'{np.random.randn():.1f}', ha='center', fontsize=8)

ax1.text(5, 2.5, 'One value for EVERY\n(state, action) pair', ha='center', fontsize=11)
ax1.text(5, 1.5, '❌ Cannot handle large/continuous states', ha='center', fontsize=10, color='#d32f2f')
ax1.text(5, 0.8, '❌ No generalization to new states', ha='center', fontsize=10, color='#d32f2f')

# Right: Function approximation (generalization)
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.set_title('FUNCTION APPROXIMATION\n"Generalization"', fontsize=14, fontweight='bold', color='#388e3c')

# Draw a neural network
layer_sizes = [4, 5, 4, 2]
layer_x = [2, 4, 6, 8]
colors = ['#bbdefb', '#fff3e0', '#fff3e0', '#c8e6c9']

for l, (x, size, color) in enumerate(zip(layer_x, layer_sizes, colors)):
    for n in range(size):
        y = 5 + (n - (size-1)/2) * 0.8
        circle = Circle((x, y), 0.25, facecolor=color, edgecolor='black', linewidth=1.5)
        ax2.add_patch(circle)
        
        # Draw connections to next layer
        if l < len(layer_sizes) - 1:
            next_size = layer_sizes[l+1]
            for n2 in range(next_size):
                y2 = 5 + (n2 - (next_size-1)/2) * 0.8
                ax2.plot([x+0.25, layer_x[l+1]-0.25], [y, y2], 
                        'gray', linewidth=0.3, alpha=0.5)

# Labels
ax2.text(2, 3.2, 'State', ha='center', fontsize=10)
ax2.text(5, 3.2, 'Hidden Layers', ha='center', fontsize=10)
ax2.text(8, 3.2, 'Q-values', ha='center', fontsize=10)

ax2.text(5, 2.2, 'Neural network LEARNS patterns', ha='center', fontsize=11)
ax2.text(5, 1.5, '✓ Works with any state representation', ha='center', fontsize=10, color='#388e3c')
ax2.text(5, 0.8, '✓ Generalizes to unseen states!', ha='center', fontsize=10, color='#388e3c')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("THE FUNDAMENTAL SHIFT")
print("="*70)
print("""
TABULAR (Q-Table):
  Q: (state, action) → lookup in table → value
  "What value did I store for this exact state-action pair?"

FUNCTION APPROXIMATION:
  Q: (state, action) → neural network → value
  "Based on patterns I've learned, what should this value be?"

The key insight: Similar states should have similar values!
Neural networks naturally learn this similarity structure.
""")
print("="*70)

---
## The Curse of Dimensionality

Why exactly do tables fail? It's a numbers game:

```
    ┌────────────────────────────────────────────────────────────────┐
    │              THE CURSE OF DIMENSIONALITY                       │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  NUMBER OF STATES = (values per dimension)^(number of dims)   │
    │                                                                │
    │  Example: Each dimension can have 10 values                   │
    │                                                                │
    │  Dims    States          Memory Needed                        │
    │  ─────────────────────────────────────────────                │
    │    1     10              80 bytes                             │
    │    2     100             800 bytes                            │
    │    3     1,000           8 KB                                 │
    │    4     10,000          80 KB                                │
    │    5     100,000         800 KB                               │
    │   10     10 billion      80 GB ← Already impractical!        │
    │   20     10^20           10^12 GB ← More than all data!      │
    │                                                                │
    │  REAL-WORLD EXAMPLES:                                         │
    │                                                                │
    │  CartPole (4 continuous dims): Infinite states!              │
    │  Atari (210×160×3 = 100,800 dims): 256^100800 states!        │
    │  Robotics (joint angles, velocities): Continuous!            │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize the exponential growth
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Exponential growth
ax1 = axes[0]
dimensions = np.arange(1, 15)
states_per_dim = 10
total_states = states_per_dim ** dimensions

ax1.semilogy(dimensions, total_states, 'b-o', linewidth=2, markersize=8)
ax1.axhline(y=1e6, color='orange', linestyle='--', linewidth=2, label='1 million states')
ax1.axhline(y=1e9, color='red', linestyle='--', linewidth=2, label='1 billion states')
ax1.axhline(y=1e12, color='darkred', linestyle='--', linewidth=2, label='1 trillion states')

ax1.fill_between(dimensions, total_states, 1, alpha=0.2, color='blue')

ax1.set_xlabel('Number of State Dimensions', fontsize=12)
ax1.set_ylabel('Total States (log scale)', fontsize=12)
ax1.set_title('The Curse of Dimensionality\n(10 values per dimension)', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(dimensions)

# Right: Real-world comparison
ax2 = axes[1]
problems = ['4×4 Grid', '10×10 Grid', 'CartPole\n(discretized)', 'Atari\n(one frame)', 'Robot\nArm']
states = [16, 100, 10000, 1e10, np.inf]
colors = ['#4caf50', '#8bc34a', '#ffeb3b', '#ff9800', '#f44336']

# Use log scale, cap infinity
states_display = [16, 100, 10000, 1e10, 1e15]
bars = ax2.bar(problems, states_display, color=colors, edgecolor='black', linewidth=2)
ax2.set_yscale('log')

# Annotations
annotations = ['Easy!\n(table)', 'OK\n(table)', 'Hard\n(need FA)', 'Impossible\n(need FA)', 'Infinite\n(need FA)']
for bar, ann in zip(bars, annotations):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height * 2, ann, 
             ha='center', fontsize=9, fontweight='bold')

ax2.set_ylabel('Number of States (log scale)', fontsize=12)
ax2.set_title('State Space Size by Problem', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nReal-world RL problems have HUGE or CONTINUOUS state spaces!")
print("We CANNOT use tables. We MUST use function approximation.")

---
## The Solution: Q(s, a) ≈ Q(s, a; θ)

Instead of storing Q-values, we **approximate** them with a function:

```
    ┌────────────────────────────────────────────────────────────────┐
    │              FROM TABLE TO FUNCTION                            │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Q-TABLE:                                                     │
    │    Q(state, action) = lookup[state][action]                   │
    │    Memory: O(|States| × |Actions|) ← HUGE!                    │
    │                                                                │
    │  FUNCTION APPROXIMATION:                                      │
    │    Q(state, action) ≈ f(state, action; θ)                    │
    │    Memory: O(|Parameters|) ← Much smaller!                    │
    │                                                                │
    │  Where θ are learnable parameters (weights and biases)        │
    │                                                                │
    │  TYPES OF FUNCTION APPROXIMATORS:                             │
    │                                                                │
    │  1. LINEAR: Q(s,a) = θᵀ × features(s,a)                       │
    │     Simple but limited expressiveness                         │
    │                                                                │
    │  2. NEURAL NETWORK: Q(s,a) = NN(s; θ)[a]                     │
    │     Powerful, can learn any function!                         │
    │     This is what DQN uses!                                    │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import torch
import torch.nn as nn

# Visualize the function approximation concept
fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')

ax.text(7, 9.5, 'Function Approximation: The Core Idea', ha='center', fontsize=16, fontweight='bold')

# Input
input_box = FancyBboxPatch((0.5, 4), 3, 3, boxstyle="round,pad=0.1",
                            facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=3)
ax.add_patch(input_box)
ax.text(2, 6.5, 'STATE', ha='center', fontsize=12, fontweight='bold', color='#1976d2')
ax.text(2, 5.8, 's = [s₁, s₂, ..., sₙ]', ha='center', fontsize=10)
ax.text(2, 5.0, 'e.g., position,', ha='center', fontsize=9, color='#666')
ax.text(2, 4.4, 'velocity, angle...', ha='center', fontsize=9, color='#666')

# Function approximator
func_box = FancyBboxPatch((4.5, 3), 5, 5, boxstyle="round,pad=0.1",
                           facecolor='#fff3e0', edgecolor='#f57c00', linewidth=3)
ax.add_patch(func_box)
ax.text(7, 7.5, 'FUNCTION', ha='center', fontsize=12, fontweight='bold', color='#f57c00')
ax.text(7, 7.0, 'APPROXIMATOR', ha='center', fontsize=12, fontweight='bold', color='#f57c00')
ax.text(7, 6.0, 'f(s; θ)', ha='center', fontsize=14, style='italic')
ax.text(7, 5.0, 'θ = learnable', ha='center', fontsize=10)
ax.text(7, 4.4, 'parameters', ha='center', fontsize=10)
ax.text(7, 3.6, '(weights & biases)', ha='center', fontsize=9, color='#666')

# Output
output_box = FancyBboxPatch((10.5, 4), 3, 3, boxstyle="round,pad=0.1",
                             facecolor='#e8f5e9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(output_box)
ax.text(12, 6.5, 'Q-VALUES', ha='center', fontsize=12, fontweight='bold', color='#388e3c')
ax.text(12, 5.7, 'Q(s, a₁)', ha='center', fontsize=10)
ax.text(12, 5.1, 'Q(s, a₂)', ha='center', fontsize=10)
ax.text(12, 4.5, '...', ha='center', fontsize=10)

# Arrows
ax.annotate('', xy=(4.3, 5.5), xytext=(3.6, 5.5),
            arrowprops=dict(arrowstyle='->', lw=3, color='#1976d2'))
ax.annotate('', xy=(10.3, 5.5), xytext=(9.6, 5.5),
            arrowprops=dict(arrowstyle='->', lw=3, color='#388e3c'))

# Benefits at bottom
ax.text(7, 1.8, 'BENEFITS:', ha='center', fontsize=12, fontweight='bold')
ax.text(7, 1.2, '✓ Compact: Few parameters vs. millions of states', ha='center', fontsize=10)
ax.text(7, 0.6, '✓ Generalizes: Similar states → Similar values', ha='center', fontsize=10)
ax.text(7, 0.0, '✓ Continuous: Works with any state representation', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## Building a Q-Network

A Q-Network takes a state as input and outputs Q-values for all actions:

```
    ┌────────────────────────────────────────────────────────────────┐
    │                   Q-NETWORK ARCHITECTURE                       │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Input: State vector [s₁, s₂, s₃, s₄]                         │
    │                ↓                                               │
    │  Hidden Layer 1: Linear(4 → 64) + ReLU                        │
    │                ↓                                               │
    │  Hidden Layer 2: Linear(64 → 64) + ReLU                       │
    │                ↓                                               │
    │  Output: Linear(64 → num_actions) → [Q(s,a₁), Q(s,a₂), ...]   │
    │                                                                │
    │  WHY THIS ARCHITECTURE:                                       │
    │  • Input layer: One neuron per state dimension                │
    │  • Hidden layers: Learn complex patterns                      │
    │  • Output layer: One Q-value per action                       │
    │  • No activation on output: Q can be any real number          │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
class QNetwork(nn.Module):
    """
    Simple Q-Network for function approximation.
    
    This replaces the Q-table with a neural network!
    
    Input: State (e.g., 4 dimensions for CartPole)
    Output: Q-value for each action (e.g., 2 for left/right)
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super().__init__()
        
        # The neural network layers
        self.network = nn.Sequential(
            # Input → Hidden 1
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            
            # Hidden 1 → Hidden 2
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            
            # Hidden 2 → Output (Q-values)
            nn.Linear(hidden_dim, action_dim)
            # NOTE: No activation! Q-values can be any number
        )
    
    def forward(self, state):
        """
        Forward pass: state → Q-values for all actions.
        
        Args:
            state: Tensor of shape (batch_size, state_dim)
        Returns:
            Q-values: Tensor of shape (batch_size, action_dim)
        """
        return self.network(state)


# Create and examine a Q-network for CartPole
state_dim = 4   # CartPole has 4 state dimensions
action_dim = 2  # CartPole has 2 actions (left, right)

q_net = QNetwork(state_dim, action_dim)

print("Q-NETWORK FOR CARTPOLE")
print("="*60)
print(f"\nNetwork architecture:")
print(q_net)

# Count parameters
total_params = sum(p.numel() for p in q_net.parameters())
print(f"\nTotal parameters: {total_params:,}")

# Compare to tabular
discretized_states = 1000
table_size = discretized_states * action_dim
print(f"\nCOMPARISON:")
print(f"  Q-Network: {total_params:,} parameters")
print(f"  Q-Table (1000 discretized states): {table_size:,} values")
print(f"  Ratio: Network is ~{table_size/total_params:.1f}x smaller!")
print("\nAnd the network handles CONTINUOUS states!")
print("="*60)

In [ ]:
# Demonstrate how the Q-network works

print("HOW THE Q-NETWORK WORKS")
print("="*60)

# Sample states (like CartPole observations)
sample_states = [
    [0.0, 0.0, 0.0, 0.0],    # Perfectly balanced
    [0.5, 0.1, 0.1, 0.2],    # Slightly tilted right
    [-0.5, -0.1, -0.1, -0.2] # Slightly tilted left
]

print("\nState format: [cart_pos, cart_vel, pole_angle, pole_vel]\n")
print(f"{'State':<40} {'Q(LEFT)':<12} {'Q(RIGHT)':<12} {'Best':<10}")
print("-"*75)

for state in sample_states:
    state_tensor = torch.FloatTensor(state).unsqueeze(0)
    
    with torch.no_grad():
        q_values = q_net(state_tensor)
    
    q_left = q_values[0, 0].item()
    q_right = q_values[0, 1].item()
    best = "LEFT" if q_left > q_right else "RIGHT"
    
    print(f"{str(state):<40} {q_left:<12.4f} {q_right:<12.4f} {best:<10}")

print("\n" + "-"*75)
print("NOTE: These are random values before training!")
print("After training, similar states will have similar Q-values.")
print("="*60)

---
## Training with Gradient Descent

How do we train the Q-network? We minimize the TD error using gradient descent!

```
    ┌────────────────────────────────────────────────────────────────┐
    │              TRAINING THE Q-NETWORK                            │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  RECALL Q-LEARNING UPDATE (tabular):                          │
    │    Q(s,a) ← Q(s,a) + α × [r + γ·max Q(s',a') - Q(s,a)]        │
    │                            └───────────────────────┘           │
    │                                    TD target                   │
    │                                                                │
    │  FOR NEURAL NETWORKS, WE USE GRADIENT DESCENT:                │
    │                                                                │
    │  1. Compute TD target: y = r + γ × max Q(s', a'; θ)           │
    │                                                                │
    │  2. Compute prediction: Q(s, a; θ)                            │
    │                                                                │
    │  3. Compute loss: L = (y - Q(s, a; θ))²                       │
    │                                                                │
    │  4. Update: θ ← θ - α × ∇_θ L                                 │
    │                                                                │
    │  This is the same idea as tabular, but uses backpropagation! │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
def compute_q_loss(q_net, states, actions, rewards, next_states, dones, gamma=0.99):
    """
    Compute the Q-learning loss for a batch of transitions.
    
    This is the CORE of training a Q-network!
    
    Loss = mean( (target - prediction)² )
         = mean( (r + γ·max Q(s',a') - Q(s,a))² )
    
    Args:
        q_net: The Q-network
        states: Batch of current states
        actions: Batch of actions taken
        rewards: Batch of rewards received
        next_states: Batch of resulting states
        dones: Batch of done flags (1 if terminal, 0 otherwise)
        gamma: Discount factor
    
    Returns:
        loss: The MSE loss between predictions and targets
    """
    # ========================================
    # STEP 1: Get current Q-values
    # ========================================
    # Q(s, a) for all actions
    all_q_values = q_net(states)
    # Select Q(s, a) for the actions we actually took
    q_values = all_q_values.gather(1, actions.unsqueeze(1)).squeeze()
    
    # ========================================
    # STEP 2: Compute targets
    # ========================================
    with torch.no_grad():  # Don't compute gradients for targets!
        # Q(s', a') for all actions
        next_q_values = q_net(next_states)
        # max Q(s', a')
        max_next_q = next_q_values.max(dim=1)[0]
        # Target: r + γ * max Q(s', a') * (1 - done)
        targets = rewards + gamma * max_next_q * (1 - dones)
    
    # ========================================
    # STEP 3: Compute loss
    # ========================================
    loss = nn.functional.mse_loss(q_values, targets)
    
    return loss


# Demonstrate with a sample batch
print("Q-LEARNING LOSS COMPUTATION")
print("="*60)

batch_size = 4

# Create sample batch
states = torch.randn(batch_size, state_dim)
actions = torch.randint(0, action_dim, (batch_size,))
rewards = torch.tensor([1.0, 1.0, 1.0, 0.0])  # Normal rewards + terminal
next_states = torch.randn(batch_size, state_dim)
dones = torch.tensor([0.0, 0.0, 0.0, 1.0])  # Last one is terminal

# Compute loss
loss = compute_q_loss(q_net, states, actions, rewards, next_states, dones)

print(f"\nSample batch of {batch_size} transitions:")
print(f"  States shape: {states.shape}")
print(f"  Actions: {actions.tolist()}")
print(f"  Rewards: {rewards.tolist()}")
print(f"  Dones: {dones.tolist()}")
print(f"\nComputed loss: {loss.item():.4f}")

print("\n" + "-"*60)
print("The loss measures how far our Q-predictions are from targets.")
print("Training minimizes this loss, making predictions more accurate!")
print("="*60)

In [ ]:
# Demonstrate a single training step

print("SINGLE TRAINING STEP")
print("="*60)

# Create optimizer
optimizer = torch.optim.Adam(q_net.parameters(), lr=0.001)

# Record Q-value before training
test_state = torch.FloatTensor([[0.1, 0.0, 0.05, 0.0]])
with torch.no_grad():
    q_before = q_net(test_state).numpy()[0]

print(f"\nQ-values BEFORE training:")
print(f"  Q(LEFT): {q_before[0]:.4f}")
print(f"  Q(RIGHT): {q_before[1]:.4f}")

# Perform multiple training steps
n_steps = 100
for _ in range(n_steps):
    # Generate fake batch
    states = torch.randn(32, state_dim)
    actions = torch.randint(0, action_dim, (32,))
    rewards = torch.ones(32)
    next_states = torch.randn(32, state_dim)
    dones = torch.zeros(32)
    
    # Compute loss
    loss = compute_q_loss(q_net, states, actions, rewards, next_states, dones)
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Record Q-value after training
with torch.no_grad():
    q_after = q_net(test_state).numpy()[0]

print(f"\nQ-values AFTER {n_steps} training steps:")
print(f"  Q(LEFT): {q_after[0]:.4f}")
print(f"  Q(RIGHT): {q_after[1]:.4f}")

print(f"\nChange:")
print(f"  Q(LEFT): {q_after[0] - q_before[0]:+.4f}")
print(f"  Q(RIGHT): {q_after[1] - q_before[1]:+.4f}")

print("\n" + "-"*60)
print("The Q-values changed because of gradient descent!")
print("With proper training data, they would converge to true values.")
print("="*60)

---
## The Deadly Triad: Why This Is Dangerous

Combining three things can cause training to **diverge** (explode!):

```
    ┌────────────────────────────────────────────────────────────────┐
    │                   THE DEADLY TRIAD                             │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │           ┌───────────────┐                                   │
    │           │    FUNCTION   │                                   │
    │           │ APPROXIMATION │                                   │
    │           └───────┬───────┘                                   │
    │                   │                                           │
    │         DANGER    │                                           │
    │           ZONE    ▼                                           │
    │   ┌───────────────────────────────┐                           │
    │   │                               │                           │
    │   │    COMBINING ALL THREE        │                           │
    │   │     CAN CAUSE DIVERGENCE!     │                           │
    │   │                               │                           │
    │   └───────────────────────────────┘                           │
    │         ▲                   ▲                                 │
    │         │                   │                                 │
    │   ┌─────┴─────┐       ┌─────┴─────┐                           │
    │   │BOOTSTRAPPING│     │ OFF-POLICY │                          │
    │   │ (TD Learning)│    │ (Q-Learning)│                          │
    │   └───────────┘       └───────────┘                           │
    │                                                                │
    │  WHY IT'S DANGEROUS:                                          │
    │                                                                │
    │  1. CORRELATED DATA: Sequential experiences are similar      │
    │     → Network overfits to recent experiences                  │
    │                                                                │
    │  2. NON-STATIONARY TARGETS: Targets depend on Q-network      │
    │     → When network updates, targets change too!              │
    │                                                                │
    │  3. BOOTSTRAPPING ERROR: Small errors compound over time     │
    │     → Errors in Q(s') affect Q(s), which affects Q(s'')...   │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize the deadly triad
fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')

ax.text(7, 9.5, 'The Deadly Triad', ha='center', fontsize=18, fontweight='bold', color='#d32f2f')

# Three circles forming a triangle
positions = [(3, 6), (11, 6), (7, 2)]
labels = ['Function\nApproximation', 'Off-Policy\nLearning', 'Bootstrapping']
examples = ['(Neural Networks)', '(Q-Learning)', '(TD Learning)']
colors = ['#bbdefb', '#c8e6c9', '#fff3e0']
edges = ['#1976d2', '#388e3c', '#f57c00']

for (x, y), label, example, color, edge in zip(positions, labels, examples, colors, edges):
    circle = Circle((x, y), 1.5, facecolor=color, edgecolor=edge, linewidth=3)
    ax.add_patch(circle)
    ax.text(x, y + 0.3, label, ha='center', va='center', fontsize=11, fontweight='bold')
    ax.text(x, y - 0.5, example, ha='center', va='center', fontsize=9, color='#666')

# Danger zone in center
danger_circle = Circle((7, 5), 1.2, facecolor='#ffcdd2', edgecolor='#d32f2f', linewidth=3)
ax.add_patch(danger_circle)
ax.text(7, 5.3, '⚠️', ha='center', fontsize=24)
ax.text(7, 4.6, 'DIVERGENCE', ha='center', fontsize=9, fontweight='bold', color='#d32f2f')

# Lines connecting
for (x1, y1), (x2, y2) in [(positions[0], positions[1]), 
                           (positions[1], positions[2]),
                           (positions[2], positions[0])]:
    ax.plot([x1, x2], [y1, y2], 'r--', linewidth=2, alpha=0.5)

# Solutions at bottom
ax.text(7, -0.5, 'SOLUTIONS (coming in next notebooks!):', 
        ha='center', fontsize=12, fontweight='bold', color='#388e3c')
ax.text(4, -1.2, '• Experience Replay', ha='center', fontsize=10)
ax.text(7, -1.2, '• Target Networks', ha='center', fontsize=10)
ax.text(10, -1.2, '• Careful tuning', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nDQN solves this with two key innovations:")
print("1. EXPERIENCE REPLAY: Breaks correlation in training data")
print("2. TARGET NETWORK: Stabilizes the targets")

---
## Summary: Key Takeaways

### Why Function Approximation?

| Problem | Tabular | Function Approx |
|---------|---------|----------------|
| Small discrete states | ✓ Works | ✓ Works |
| Large discrete states | ✗ Too much memory | ✓ Works |
| Continuous states | ✗ Impossible | ✓ Works |
| Generalization | ✗ None | ✓ Similar states → Similar values |

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Q-Network** | Neural network that approximates Q(s, a) |
| **Gradient Descent** | Minimize (target - prediction)² |
| **Deadly Triad** | FA + Bootstrapping + Off-policy = Danger |

### The Core Equation

```
Loss = (r + γ × max Q(s', a'; θ) - Q(s, a; θ))²
        └─────────────────────────────────────┘
                    TD Error squared
```

---
## Test Your Understanding

**1. Why can't we use a Q-table for continuous state spaces?**
<details>
<summary>Click to reveal answer</summary>
Continuous state spaces have infinitely many possible states. You can't create a table row for every possible real-valued state. For example, a position of 1.00001 is different from 1.00002, and there are infinitely many values between them. Function approximation maps these continuous states to Q-values using a finite number of parameters.
</details>

**2. What are the benefits of function approximation over tables?**
<details>
<summary>Click to reveal answer</summary>
1. **Generalization**: Similar states get similar Q-values automatically. The network learns patterns, not individual states.
2. **Compact**: Far fewer parameters than states. A network with 1000 parameters can handle billions of states.
3. **Continuous**: Works with any state representation - no discretization needed.
</details>

**3. What is the "deadly triad" and why is it dangerous?**
<details>
<summary>Click to reveal answer</summary>
The deadly triad is the combination of:
1. Function approximation (neural networks)
2. Bootstrapping (TD learning - using estimates to update estimates)
3. Off-policy learning (learning about a different policy than we follow)

It's dangerous because: training data is correlated (sequential), targets are non-stationary (depend on the changing network), and small errors can compound. This can cause training to diverge!
</details>

**4. How do we train a Q-network?**
<details>
<summary>Click to reveal answer</summary>
1. Compute the TD target: y = r + γ × max Q(s', a')
2. Compute the prediction: Q(s, a)
3. Compute the loss: (y - Q(s, a))²
4. Update weights using gradient descent: θ ← θ - α × ∇L

This is the same as tabular Q-learning, but using backpropagation!
</details>

**5. What solutions does DQN use for the deadly triad?**
<details>
<summary>Click to reveal answer</summary>
DQN uses two key innovations:
1. **Experience Replay**: Store experiences in a buffer and sample randomly. This breaks the correlation between consecutive experiences.
2. **Target Network**: Use a separate, slowly-updated network for computing targets. This stabilizes the targets during training.
</details>

---
## What's Next?

You now understand why we need function approximation and the challenges it brings!

In the next notebook, we'll build a complete **DQN from scratch** that solves these challenges.

**Continue to:** [Notebook 2: DQN From Scratch](02_dqn_from_scratch.ipynb)

---

*Function approximation is the bridge from toy problems to real-world RL!*